# Часть 1. Классическое машинное обучение для решения проблемы классификации текста

In [2]:
import numpy as np
import pandas as pd
import textwrap
import os
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import wordnet, stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
import string
import warnings
import re
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline

warnings.filterwarnings("ignore")

In [3]:
nltk.download("punkt")
nltk.download("wordnet")
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\timts\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\timts\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\timts\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Поспользуемся классом BbcDataset из лекции, удалим лишние методы и добавим удаление стоп слов

In [5]:
DATA_DIR = "./bbc/"

class BbcDataset:
    def __init__(self, data_dir=DATA_DIR):
        self.data_dir = data_dir
        self.documents = [
            os.path.join(path, name)
            for path, _, files in os.walk(self.data_dir)
            for name in files
        ]
        self.lemmatizer = WordNetLemmatizer()
        self.stop_words = set(stopwords.words('english'))
        self.dataframe = self.get_pandas_alike_dataset()

    def preprocess_text(self, text: str) -> str:
        """
        Удаляет все, кроме английских букв нижнего регистра и пробелов, затем лемматизирует

        :param text: Текст для обработки
        :return: Обработанная строка текста
        """
        text = re.sub(r"[^a-z\s]", "", text.lower())
        tokens = word_tokenize(text)
        tokens = [self.lemmatizer.lemmatize(word) for word in tokens]
        tokens = [word for word in tokens if word not in self.stop_words]
        return " ".join(tokens)

    def get_dataset(self):
        """Загружает все документы и их категории"""
        data = []
        for file in self.documents:
            label = os.path.basename(os.path.dirname(file))
            try:
                with open(file, "r", encoding="utf-8") as f:
                    content = f.read().strip()
                data.append((content, label))
            except Exception as e:
                print(f"Ошибка чтения файла {file}: {e}")
        return data

    def get_pandas_alike_dataset(self):
        """Возвращает DataFrame с текстами и метками"""
        data = self.get_dataset()
        return pd.DataFrame(data, columns=["text", "label"])

Создаём датасет

In [7]:
def prepare_data():
    bbc = BbcDataset()
    data = bbc.dataframe
    data['text'] = data['text'].apply(lambda x: bbc.preprocess_text(x))
    return data

In [8]:
data = prepare_data()
data

Ошибка чтения файла ./bbc/sport\199.txt: 'utf-8' codec can't decode byte 0xa3 in position 257: invalid start byte


,text,label
0,ad sale boost time warner profit quarterly pro...,business
1,dollar gain greenspan speech dollar ha hit hig...,business
2,yukos unit buyer face loan claim owner embattl...,business
3,high fuel price hit ba profit british airway h...,business
4,pernod takeover talk lift domecq share uk drin...,business
...,...,...
2215,bt program beat dialler scam bt introducing tw...,tech
2216,spam email tempt net shopper computer user acr...,tech
2217,careful code new european directive could put ...,tech
2218,u cyber security chief resigns man making sure...,tech


Делим наш датает на трейн тест выборку, не забываем указать stratify

In [10]:
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['label'], test_size=0.2, random_state=7, stratify=data['label'])

In [11]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1776,), (444,), (1776,), (444,))

Для части с классическим машинным обучением воспользуемся TF-IDF векторайзером, а модель будем использовать catboost

In [13]:
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Обучение модели классификации
classifier = CatBoostClassifier()
classifier.fit(X_train_tfidf, y_train)

# Предсказание категорий для тестовых данных
y_pred = classifier.predict(X_test_tfidf)

Learning rate set to 0.081479
0:	learn: 1.5246673	total: 635ms	remaining: 10m 34s
1:	learn: 1.4456486	total: 1.02s	remaining: 8m 28s
2:	learn: 1.3861521	total: 1.46s	remaining: 8m 6s
3:	learn: 1.3142052	total: 1.88s	remaining: 7m 48s
4:	learn: 1.2580692	total: 2.33s	remaining: 7m 42s
5:	learn: 1.1942783	total: 2.74s	remaining: 7m 33s
6:	learn: 1.1580177	total: 3.12s	remaining: 7m 22s
7:	learn: 1.1219673	total: 3.51s	remaining: 7m 15s
8:	learn: 1.0859545	total: 3.96s	remaining: 7m 16s
9:	learn: 1.0496626	total: 4.38s	remaining: 7m 13s
10:	learn: 1.0100361	total: 4.8s	remaining: 7m 11s
11:	learn: 0.9790937	total: 5.22s	remaining: 7m 9s
12:	learn: 0.9503637	total: 5.67s	remaining: 7m 10s
13:	learn: 0.9161128	total: 6.07s	remaining: 7m 7s
14:	learn: 0.8923765	total: 6.5s	remaining: 7m 6s
15:	learn: 0.8673045	total: 6.93s	remaining: 7m 6s
16:	learn: 0.8443809	total: 7.33s	remaining: 7m 3s
17:	learn: 0.8240530	total: 7.73s	remaining: 7m 1s
18:	learn: 0.8061778	total: 8.11s	remaining: 6m 58s


Смотрим результат нашего классического решения по классифиакции новостей, получилось достаточно неплохо, точность 96%, учитывая, что модель обучается быстро, а результат достаточно хороший можно сделать вывод, что это вполне достойное и недорогое решение классификкации.

In [15]:
print(classification_report(y_test, y_pred))

               precision    recall  f1-score   support

     business       0.93      0.96      0.95       101
entertainment       0.99      0.96      0.97        77
     politics       0.96      0.95      0.96        84
        sport       0.97      1.00      0.99       102
         tech       0.97      0.94      0.96        80

     accuracy                           0.96       444
    macro avg       0.97      0.96      0.96       444
 weighted avg       0.96      0.96      0.96       444



In [16]:
def classifier_prediction(input):
    return classifier.predict(tfidf_vectorizer.transform([input]))[0][0]

# Часть 2. Использование DistilBert для решения проблемы классификации текста

In [18]:
import torch
import os
import transformers # pip install transformers чтобы установить библиотеку для работы с моделями-трансформерами
import numpy as np
import sys
import matplotlib.pyplot as plt
import warnings
import pandas as pd
torch.backends.cudnn.deterministic = True
warnings.filterwarnings("ignore")

Создаём новый датасет без предобработки

In [20]:
bbc_1 = BbcDataset()
data_1 = bbc_1.dataframe

Ошибка чтения файла ./bbc/sport\199.txt: 'utf-8' codec can't decode byte 0xa3 in position 257: invalid start byte


In [21]:
X_train, X_test, y_train, y_test = train_test_split(data_1['text'], data_1['label'], test_size=0.2, random_state=7, stratify=data_1['label'])

In [22]:
train_texts = X_train.values
train_labels = y_train.values

test_texts = X_test.values
test_labels = y_test.values

Используем предобученный токенизатор

In [24]:
tokenizer = transformers.DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [25]:
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True)

In [26]:
class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx]).long()
        return item

    def __len__(self):
        return len(self.labels)

Кодируем лэйблы

In [28]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train_encoded_labels = le.fit_transform(train_labels)
test_encoded_labels = le.transform(test_labels)

In [29]:
label_decode = {
    0: 'business',
    1: 'entertainment',
    2: 'politics',
    3: 'sport',
    4: 'tech',
}

In [30]:
train_dataset = IMDbDataset(train_encodings, train_encoded_labels)
test_dataset = IMDbDataset(test_encodings, test_encoded_labels)

In [31]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False)

Грузим предобученный DistilBert для классификации последовательностей, указываем количество лейблов = 5

In [33]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("DEVICE: ", DEVICE)
model = transformers.DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=5)
model.to(DEVICE)
model.train()

optim = torch.optim.Adam(model.parameters(), lr=5e-5)

DEVICE:  cpu


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [34]:
def compute_accuracy(model, data_loader, device):
    with torch.no_grad():
        correct_pred, num_examples = 0, 0

        for batch_idx, batch in enumerate(data_loader):

        ### Prepare data
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs['logits']
            predicted_labels = torch.argmax(logits, 1)
            num_examples += labels.size(0)
            correct_pred += (predicted_labels == labels).sum()

        return correct_pred.float()/num_examples * 100

In [35]:
import time
start_time = time.time()
num_epochs = 3

for epoch in range(num_epochs):

    model.train()

    for batch_idx, batch in enumerate(train_loader):

        ### Prepare data
        input_ids = batch['input_ids'].to(DEVICE)
        attention_mask = batch['attention_mask'].to(DEVICE)
        labels = batch['labels'].to(DEVICE)

        ### Forward
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss, logits = outputs['loss'], outputs['logits']

        ### Backward
        optim.zero_grad()
        loss.backward()
        optim.step()

        ### Logging
        if not batch_idx % 50:
            print (f'Epoch: {epoch+1:04d}/{num_epochs:04d} | '
                   f'Batch {batch_idx:04d}/{len(train_loader):04d} | '
                   f'Loss: {loss:.4f}')

    model.eval()

    with torch.set_grad_enabled(False):
        print(f'Training accuracy: '
              f'{compute_accuracy(model, train_loader, DEVICE):.2f}%'
              f'\nTest accuracy: '
              f'{compute_accuracy(model, test_loader, DEVICE):.2f}%')

    print(f'Time elapsed: {(time.time() - start_time)/60:.2f} min')

print(f'Total Training Time: {(time.time() - start_time)/60:.2f} min')
print(f'Test accuracy: {compute_accuracy(model, test_loader, DEVICE):.2f}%')

Epoch: 0001/0003 | Batch 0000/0111 | Loss: 1.5786
Epoch: 0001/0003 | Batch 0050/0111 | Loss: 0.1371
Epoch: 0001/0003 | Batch 0100/0111 | Loss: 0.0218
Training accuracy: 98.42%
Test accuracy: 96.17%
Time elapsed: 39.98 min
Epoch: 0002/0003 | Batch 0000/0111 | Loss: 0.0229
Epoch: 0002/0003 | Batch 0050/0111 | Loss: 0.0267
Epoch: 0002/0003 | Batch 0100/0111 | Loss: 0.0719
Training accuracy: 99.16%
Test accuracy: 97.75%
Time elapsed: 79.03 min
Epoch: 0003/0003 | Batch 0000/0111 | Loss: 0.0235
Epoch: 0003/0003 | Batch 0050/0111 | Loss: 0.0081
Epoch: 0003/0003 | Batch 0100/0111 | Loss: 0.0048
Training accuracy: 99.27%
Test accuracy: 97.75%
Time elapsed: 118.07 min
Total Training Time: 118.07 min
Test accuracy: 97.75%


Пишем функцию для классификации текста на основе модели

In [37]:
def predict(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    outputs = model(**inputs)
    predicted_class = torch.argmax(outputs.logits, dim=-1).item()
    return label_decode[predicted_class]

Сохраняем модель

In [39]:
output_dir = './model_save/'


if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

Saving model to ./model_save/


('./model_save/tokenizer_config.json',
 './model_save/special_tokens_map.json',
 './model_save/vocab.txt',
 './model_save/added_tokens.json',
 './model_save/tokenizer.json')

# Предворительные выводы, после проведения обучения.
Итоговое качество модели Берт оказались выше (98% против 96% у модели классического машинного обучения на тестовых выборках), однако нужно отметить несопоставимую разницу в скорости обучения и скорость работы в целом. Класическое решение учится в разы быстрее и незначительно хуже результат, потому для конкретно этой задачи лучше использовать классический подход, если бы для заказчика полученный уровень качества оказался бы достаточным. Хотя если бы расчёты производились бы на cuda, то обучение нейронной сети прошло быстрее, но опять же это ресурсы, которых может и не быть. С другой стороны, для классического машинного обучения понадобилось больше подготовительных действий: очитска и обработка датасета, в то время как нейронной сети мы просто отдали размеченный датасет.

# Проведём тестирование моделей на актуальной сохранённой выборке

Я скопирую класс BBCdataset для его небольшой корректировки, я хочу добавить обработку урлов в датасет

In [155]:
DATA_DIR = "./saved_news/"


class BbcDatasetForTesting:
    def __init__(self, data_dir=DATA_DIR):
        self.data_dir = data_dir
        self.documents = [
            os.path.join(path, name)
            for path, _, files in os.walk(self.data_dir)
            for name in files
        ]
        self.lemmatizer = WordNetLemmatizer()
        self.stop_words = set(stopwords.words('english'))
        self.dataframe = self.get_pandas_alike_dataset()

    def preprocess_text(self, text: str) -> str:
        """
        Удаляет все, кроме английских букв нижнего регистра и пробелов, затем лемматизирует

        :param text: Текст для обработки
        :return: Обработанная строка текста
        """
        text = re.sub(r"[^a-z\s]", "", text.lower())
        tokens = word_tokenize(text)
        tokens = [self.lemmatizer.lemmatize(word) for word in tokens]
        tokens = [word for word in tokens if word not in self.stop_words]
        return " ".join(tokens)

    def get_dataset(self):
        """Загружает все документы и их категории"""
        data = []
        for file in self.documents:
            label = os.path.basename(os.path.dirname(file))
            try:
                with open(file, "r", encoding="utf-8") as f:
                    content = f.read().strip()
                    url = re.findall(r'''http[\:/a-zа-яA-ZА-Я0-9\.\?\=&-]*''', content)[0]
                    content = re.sub(r'''http[\:/a-zа-яA-ZА-Я0-9\.\?\=&-]*''', '', content)
                data.append((content, label, url))
            except Exception as e:
                print(f"Ошибка чтения файла {file}: {e}")
        return data

    def get_pandas_alike_dataset(self):
        """Возвращает DataFrame с текстами и метками"""
        data = self.get_dataset()
        return pd.DataFrame(data, columns=["text", "label", 'url'])

А теперь проведём предобработку, предскажем метки и сохраним всё в один датасет, проведём визуальный анализ

In [158]:
bbc_valid_data = BbcDatasetForTesting()

In [160]:
valid_data = bbc_valid_data.get_pandas_alike_dataset()
valid_data['text'] = valid_data['text'].apply(lambda x: bbc_valid_data.preprocess_text(x))

In [161]:
valid_data['ML_prediction'] = valid_data['text'].apply(classifier_prediction)

In [162]:
valid_data['DistilBert_predictiom'] = valid_data['text'].apply(predict)

In [164]:
valid_data = valid_data.reindex(columns=['text', 'label', 'ML_prediction', 'DistilBert_predictiom', 'url'])

In [165]:
valid_data

,text,label,ML_prediction,DistilBert_predictiom,url
0,adrien brody score olivier nomination oscar wi...,arts,entertainment,entertainment,https://www.bbc.com/news/articles/cy9dx2g9pplo
1,something stirring beneath surface eight ghost...,arts,sport,tech,https://www.bbc.com/culture/article/20250221-e...
2,im proud wonderful teen amy heckerling clueles...,arts,entertainment,tech,https://www.bbc.com/culture/article/20250225-a...
3,stirred people breathless wonder scalding abus...,arts,politics,tech,https://www.bbc.com/culture/article/20250221-t...
4,rare blue maya invented museo nacional de arte...,arts,sport,tech,https://www.bbc.com/culture/article/20180816-t...
5,uk keep option open trump tariff kick getty im...,business,business,business,https://www.bbc.com/news/articles/cx2r3md0j84o
6,u education department plan cut half workforce...,business,politics,business,https://www.bbc.com/news/articles/c62zl638m71o
7,trump mulling mineral deal conflicthit dr cong...,business,business,business,https://www.bbc.com/news/articles/cjryjlndddpo
8,trump halt plan steel aluminium tariff canada ...,business,business,business,https://www.bbc.com/news/articles/cm2y811g1dgo
9,european stock steady u market plunge watch tr...,business,business,business,https://www.bbc.com/news/articles/c4gdwgjkk1no


По итогу получилась небольшая мешанина, так даже немного веселее. Дело в том, что выборка новостей с тегами устарела, теперь в сайте BBC немного другие разделы, я собрал по 5 новостей из каждого раздела, отметил реальными тегами, к которым они относятся и соотнём их с 5ю предсказанными. К сожалению Я не могу выдать вероятность модели, так как совершил небольшую ошибку, закодировав классы через label encoder. наверное если бы кодировать через one hot получился бы массив с 5ю метками и получилось бы лучше, в плане оценки вероятности. Если будет время, постараюсь исправить. Тем не менее получилось достаточно интересно и всё таки глядя на актуальные метки и то как модели справились с классификацией, наверное стоит отметить DistilBert, как модель которая справилась значительно лучше.

In [167]:
def make_clickable_both(val): 
    return f'<a href="{val}">Ссылка</a>'

valid_data.style.format({'url': make_clickable_both})